## Algorithm 01 ##

In [3]:
def update_coordinates(frame_n, frame_n1):
    updated_frame_n = frame_n.copy()
    N = len(frame_n)

    updated_coordinates = [] 
    for n in range(N):
        class_n, xn, yn, wn, hn = frame_n[n]
        class_n1, xn1, yn1, wn1, hn1 = frame_n1[n]

        if class_n == 0 and class_n1 == 0:
            dx = abs(xn1 - xn)
            dy = abs(yn1 - yn)

            if dx < 10 and dy < 10:
                xn1, yn1 = xn, yn
            elif dx < 10 and dy >= 10:
                xn1 = xn
            elif dx >= 10 and dy < 10:
                yn1 = yn

            updated_frame_n[n] = [class_n, xn1, yn1, wn, hn]
            updated_coordinates.append(updated_frame_n[n])  

        elif class_n == 1 and class_n1 == 1:
            dx = abs(xn1 - xn)
            dy = abs(yn1 - yn)

            if dx < 10 and dy < 10:
                xn1, yn1 = xn, yn
            elif dx < 10 and dy >= 10:
                xn1 = xn
            elif dx >= 10 and dy < 10:
                yn1 = yn

            updated_frame_n[n] = [class_n, xn1, yn1, wn, hn]
            updated_coordinates.append(updated_frame_n[n]) 

    return updated_coordinates 



updated_coordinates = update_coordinates(frame_n, frame_n1)


for n, updated_coord in enumerate(updated_coordinates):
    class_n, xn, yn, wn, hn = updated_coord
    print(f"[{class_n}] {xn} {yn} {wn} {hn}")


[0] 540 320 50 60
[1] 435 256 120 30


## Algorithm 02 ##

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("classification.xlsx")

# Define the rules
def apply_rules(row):
    if row['objectness_crane'] == 1 and row['objectness_module'] == 1 and row['movement_crane'] == 1 and row['movement_module'] == 1:
        return 2
    elif row['objectness_crane'] == 1 and row['objectness_module'] == 0 and row['movement_crane'] == 1 and row['movement_module'] == 0:
        return 4
    elif row['objectness_crane'] == 1 and row['objectness_module'] == 0 and row['movement_crane'] == 0 and row['movement_module'] == 0:
        return 5
    else:
        return 0  # For rows that don't match any rule

# Apply the rules to create the "Prediction" column
df['Prediction'] = df.apply(apply_rules, axis=1)

# Save the updated DataFrame back to the Excel file
df.to_excel("classification_with_prediction.xlsx", index=False, engine='openpyxl')

In [ ]:
import pandas as pd

df = pd.read_excel("classification_with_prediction.xlsx")

continuous_zeros_sequence = []  
new_predictions = [] 

for index, row in df.iterrows():
    prediction = row['Prediction']
    
    if prediction == 0:
        continuous_zeros_sequence.append(index)
    else:
        if continuous_zeros_sequence:
            if df.at[continuous_zeros_sequence[0] - 1, 'Prediction'] == 2 and df.at[index, 'Prediction'] == 4:
                for seq_index in continuous_zeros_sequence:
                    new_predictions.append(3)
            elif df.at[continuous_zeros_sequence[0] - 1, 'Prediction'] == 4 and df.at[index, 'Prediction'] == 2:
                for seq_index in continuous_zeros_sequence:
                    new_predictions.append(1)
            elif df.at[continuous_zeros_sequence[0] - 1, 'Prediction'] == 5 and df.at[index, 'Prediction'] == 2:
                for seq_index in continuous_zeros_sequence:
                    new_predictions.append(1)
            elif df.at[continuous_zeros_sequence[0] - 1, 'Prediction'] == 5 and df.at[index, 'Prediction'] == 0:
                for seq_index in continuous_zeros_sequence:
                    new_predictions.append(1)
            else:
                new_predictions.extend([0] * len(continuous_zeros_sequence))
            continuous_zeros_sequence = []  
        new_predictions.append(prediction)  


if continuous_zeros_sequence:
    new_predictions.extend([0] * len(continuous_zeros_sequence))

df['Prediction'] = new_predictions

df.to_excel("classification_updated.xlsx", index=False, engine='openpyxl')
